In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
 

In [ ]:
path = "fashion-mnist.csv"

df = pd.read_csv(path)
df.head()

## Funciones

Funcion para ver alguna imagen de la clase elegida

In [ ]:
def mostrarImagenDe (label: int):
    clase = df[df['label'] == label].drop(columns='label')
    imagen = clase.iloc[np.random.randint(0,6000)].values.reshape(28,28)
    sns.heatmap(data=imagen, cmap='gray',cbar=False, square=True,annot=False)
    plt.show()
    plt.close()

mostrarImagenDe(5)

Funcion que me devuelve la imagen promedio de una clase en particular

In [ ]:
def promedioImagen(prenda:int):
    clase = df[df['label']== prenda].drop(columns='label')
    promedio_imagen = np.mean(clase.values,axis=0)
    promedio_imagen /= len(df[df['label']  ==1])
    promedio_imagen = promedio_imagen.reshape(28,28)
    sns.heatmap(data=promedio_imagen, cmap='gray',cbar=False, xticklabels=[],yticklabels=[])
    


promedioImagen(5)

Validacion Cruzada

In [ ]:
def validacion_cruzada(model,X,Y,K):
    cv = KFold(n_splits=K,shuffle=True,random_state=42)
    scores = cross_val_score(model,X,Y, cv=cv, scoring='accuracy')
    accuracy_media = scores.mean()
    accuaracy_std = scores.std()
    #print(f'Precision media: {accuracy_media:.2f}, Desviacion estandar de la precision: {accuaracy_std:.2f}')
    return scores
#validacion_cruzada(neigh, X, Y, 5)

Testeo de modelos segun set de datos de entrenamiento

In [ ]:
def testearModelo (modelo,x,y):
    x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.2, shuffle=True) #Aca separo el dataset en train y val
    modelo.fit(x_train, y_train)
    y_predict = modelo.predict(x_val)
    return accuracy_score(y_val,y_predict)

Calibracion de modelos

In [ ]:
def calibrar(model,x,y):
    x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.2, shuffle=True)
    model.fit(x_train,y_train)
    best_param = model.best_params_
    best_model = model.best_estimator_
    accuracy = best_model.score(x_val,y_val)
    print(f"El mejor k es: {best_param}, con una precision de: {accuracy} ")

## Ejercicios

### Ejercicio 1.b ¿Hay clases de prendas que son parecidas entre si?

Cuando comparamos entre remaras y pantalones, la precision de clasificacion es mayor que cuando se intenta clasificar remeras y pullover

In [ ]:
remYpants = df[(df['label'] == 1) | (df['label'] == 0)] #dataset de remeras y pantalones
Y = remYpants['label']
X = remYpants[['pixel1','pixel196','pixel350']] #solo le damos 3 pixeles para que entrene
k = 5
x_train, x_val, y_train, y_val = train_test_split(X,Y, test_size=0.2,shuffle=True)

neigh = KNeighborsClassifier(n_neighbors=k)

print(f"La precision cuando se comparan remeras con pullovers es de : {np.mean(validacion_cruzada(neigh,X,Y,k))}")

In [ ]:
remYpull = df[(df['label']== 0) | (df['label']== 2)]
X1 = remYpull[['pixel1','pixel196','pixel350']]
Y1 = remYpull['label']

x1_train, x1_val, y1_train, y1_val = train_test_split(X1,Y1, test_size=0.2, shuffle=True)

neigh2 = KNeighborsClassifier(n_neighbors=k)

print(f"La precision cuando se comparan remeras con pullovers es de : {np.mean(validacion_cruzada(neigh2,X1,Y1,k))}")

In [ ]:
graficarScores(validacion_cruzada(neigh,X,Y,k),validacion_cruzada(neigh2,X1,Y1,k),k)


ValueError: All arrays must be of the same length

### Ejercicio 2

2.a) Creo los DataFrames de pantalones y remeras

In [ ]:
pantalones = df[df['label'] == 1].drop(columns='label')
remeras = df[df['label'] == 0].drop(columns='label')

2.b) Aca podemos ver que cada conjunto de datos de cada clase tiene la misma cantidad de muestras por lo que podemos decir que esta balanceado

In [ ]:
print("Cantidad de muestras en pantalones: ", len(pantalones))
print("Cantidad de muestras en remeras: ", len(remeras))

2.c) Propusimos distintos conjuntos de 3 atributos y 2 de 4 para comparar distintos modelos KNN. En la funcion `testearModelo()` se divide el data set en train y val y nos devuelve la precision de ese modelo 

In [ ]:
#aca propongo 5 conjunto de atributos distintos

cols1 =['pixel1','pixel2','pixel3'] #primeros 3 pixeles de la esquina superior izquierda
cols2 = ['pixel14','pixel210','pixel406'] # recta vertical en el medio de la imagen
cols3 = ['pixel1','pixel175','pixel378'] # diagonal desdes esquina superior izquierda hasta esquina inferior derecha
cols4 = ['pixel1','pixel175','pixel378','pixel581'] #diagonal con 1 pixel mas
cols5 = ['pixel14','pixel210','pixel406','pixel602'] #Recta vertical con 1 pixel mas


Xa = remYpants[cols1]
Xb = remYpants[cols2]
Xc = remYpants[cols3]
Xd = remYpants[cols4]
Xf = remYpants[cols5]
Ys = remYpants['label']

knn5 = KNeighborsClassifier(n_neighbors=5)

#Aca testeo un modelo knn de 5 vecinos con los diferentes conjuntos de test
print(f"La precision del modelo knn5 con el dataset Xa es de: {testearModelo(knn5,Xa,Ys)}")
print(f"La precision del modelo knn5 con el dataset Xa es de: {testearModelo(knn5,Xb,Ys)}")
print(f"La precision del modelo knn5 con el dataset Xa es de: {testearModelo(knn5,Xc,Ys)}")
print(f"La precision del modelo knn5 con el dataset Xa es de: {testearModelo(knn5,Xd,Ys)}")
print(f"La precision del modelo knn5 con el dataset Xa es de: {testearModelo(knn5,Xf,Ys)}")

In [ ]:
knn = KNeighborsClassifier()
hiper_grid = {'n_neighbors':[1,2,3,4,5,7,9,10,11]}

grid_search = GridSearchCV(knn, hiper_grid, cv=5, scoring='accuracy')

calibrar(grid_search,Xa,Ys)
calibrar(grid_search,Xb,Ys)
calibrar(grid_search,Xc,Ys)
calibrar(grid_search,Xd,Ys)
calibrar(grid_search,Xf,Ys)